In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Base

In [2]:
! pip install transformers[sentencepiece] datasets
! pip install sacrebleu sentencepiece
! pip install huggingface_hub
!pip install gradio
!pip install sentencepiece
!pip install langdetect

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 4.9 MB/s 
     |████████████████████████████████| 452 kB 70.7 MB/s 
     |████████████████████████████████| 182 kB 76.6 MB/s 
     |████████████████████████████████| 212 kB 62.7 MB/s 
     |████████████████████████████████| 132 kB 61.5 MB/s 
     |████████████████████████████████| 127 kB 61.8 MB/s 
     |████████████████████████████████| 7.6 MB 53.5 MB/s 
     |████████████████████████████████| 1.3 MB 52.9 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 118 kB 5.3 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: http

In [3]:
!git clone https://github.com/virsagothethird/Korean-English-translator.git

Cloning into 'Korean-English-translator'...
remote: Enumerating objects: 393, done.
remote: Counting objects: 100% (27/27), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 393 (delta 10), reused 11 (delta 2), pack-reused 366
Receiving objects: 100% (393/393), 358.37 MiB | 32.82 MiB/s, done.
Resolving deltas: 100% (166/166), done.
Checking out files: 100% (193/193), done.


# Hugging Face - Translator

In [17]:
import gradio as gr
import torch

from transformers import M2M100ForConditionalGeneration, M2M100Tokenizer

model = M2M100ForConditionalGeneration.from_pretrained("facebook/m2m100_1.2B")
tokenizer = M2M100Tokenizer.from_pretrained("facebook/m2m100_1.2B")

langs = """Afrikaans (af), Amharic (am), Arabic (ar), Asturian (ast), Azerbaijani (az), Bashkir (ba), Belarusian (be), Bulgarian (bg), Bengali (bn), Breton (br), Bosnian (bs), Catalan; Valencian (ca), Cebuano (ceb), Czech (cs), Welsh (cy), Danish (da), German (de), Greeek (el), English (en), Spanish (es), Estonian (et), Persian (fa), Fulah (ff), Finnish (fi), French (fr), Western Frisian (fy), Irish (ga), Gaelic; Scottish Gaelic (gd), Galician (gl), Gujarati (gu), Hausa (ha), Hebrew (he), Hindi (hi), Croatian (hr), Haitian; Haitian Creole (ht), Hungarian (hu), Armenian (hy), Indonesian (id), Igbo (ig), Iloko (ilo), Icelandic (is), Italian (it), Japanese (ja), Javanese (jv), Georgian (ka), Kazakh (kk), Central Khmer (km), Kannada (kn), 
Korean (ko), Luxembourgish; Letzeburgesch (lb), Ganda (lg), Lingala (ln), Lao (lo), Lithuanian (lt), Latvian (lv), Malagasy (mg), Macedonian (mk), Malayalam (ml), Mongolian (mn), Marathi (mr), Malay (ms), Burmese (my), Nepali (ne), Dutch; Flemish (nl), Norwegian (no), Northern Sotho (ns), Occitan (post 1500) (oc), Oriya (or), Panjabi; Punjabi (pa), Polish (pl), Pushto; Pashto (ps), Portuguese (pt), Romanian; Moldavian; Moldovan (ro), Russian (ru), Sindhi (sd), Sinhala; Sinhalese (si), Slovak (sk), 
Slovenian (sl), Somali (so), Albanian (sq), Serbian (sr), Swati (ss), Sundanese (su), Swedish (sv), Swahili (sw), Tamil (ta), Thai (th), Tagalog (tl), Tswana (tn), 
Turkish (tr), Ukrainian (uk), Urdu (ur), Uzbek (uz), Vietnamese (vi), Wolof (wo), Xhosa (xh), Yiddish (yi), Yoruba (yo), Chinese (zh), Zulu (zu)"""
lang_list = [lang.strip() for lang in langs.split(',')]

examples = [["Korean (ko)", "English (en)", "ddd."]]

def translate(src, tgt, text):
    src = src.split(" ")[-1][1:-1]
    tgt = tgt.split(" ")[-1][1:-1]

    # translate
    tokenizer.src_lang = src
    encoded_src = tokenizer(text, return_tensors="pt")
    generated_tokens = model.generate(**encoded_src, forced_bos_token_id=tokenizer.get_lang_id(tgt), use_cache=True)
    result = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0]
    return result



Downloading:   0%|          | 0.00/909 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.71M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/271 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.14k [00:00<?, ?B/s]

In [18]:
text = "I want to go home"
result = translate("English (en)","Korean (ko)",text)
print("[EN] ", text, " = ", "[KR] ", result)


/usr/local/lib/python3.8/dist-packages/transformers/generation/utils.py:1387: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 200 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


[EN]  I want to go home  =  [KR]  나는 집에 가고 싶어


In [20]:
text1 = "Increased size of pleural nodules in the Lt. upper lobe, R/O Pleural metastasis"
text2 = "Small amount of Lt. pleural effusion, R/O Radiation related change"
text3 = "No definite evidence of tumor recurrence in the anterior mediastinum"
text4 = "Imp : 1. R/O Pleural metastasis"
text5 = "2. No definite evidence of remained fibrosis after COVID-19 infection"
result1 = translate("English (en)","Korean (ko)",text1)
result2 = translate("English (en)","Korean (ko)",text2)
result3 = translate("English (en)","Korean (ko)",text3)
result4 = translate("English (en)","Korean (ko)",text4)
result5 = translate("English (en)","Korean (ko)",text5)

print("[EN] ", text1, "\n = ", "[KR] ", result1)
print("[EN] ", text2, "\n = ", "[KR] ", result2)
print("[EN] ", text3, "\n = ", "[KR] ", result3)
print("[EN] ", text4, "\n = ", "[KR] ", result4)
print("[EN] ", text5, "\n = ", "[KR] ", result5)


[EN]  Increased size of pleural nodules in the Lt. upper lobe, R/O Pleural metastasis 
 =  [KR]  Lt. upper lobe, R/O Pleural metastasis에 있는 pleural nodules의 크기 증가
[EN]  Small amount of Lt. pleural effusion, R/O Radiation related change 
 =  [KR]  소량의 Lt. pleural effusion, R/O 방사선 관련 변화
[EN]  No definite evidence of tumor recurrence in the anterior mediastinum 
 =  [KR]  이전 mediastinum에서 종양 재발의 확실한 증거는 없습니다.
[EN]  Imp : 1. R/O Pleural metastasis 
 =  [KR]  Imp : 1. R/O Pleural 메타스타즈
[EN]  2. No definite evidence of remained fibrosis after COVID-19 infection 
 =  [KR]  COVID-19 감염 후 남아있는 섬유증에 대한 확실한 증거는 없습니다.


# Hugging Face - FillMask


In [ ]:
from transformers import pipeline
unmasker = pipeline('fill-mask', model='emilyalsentzer/Bio_ClinicalBERT')
unmasker("Hello I'm a [MASK] model.")

Some weights of the model checkpoint at emilyalsentzer/Bio_ClinicalBERT were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


[{'score': 0.2732054889202118,
  'token': 1363,
  'token_str': 'good',
  'sequence': "hello i'm a good model."},
 {'score': 0.023285292088985443,
  'token': 7378,
  'token_str': 'linear',
  'sequence': "hello i'm a linear model."},
 {'score': 0.01701871119439602,
  'token': 2869,
  'token_str': 'poor',
  'sequence': "hello i'm a poor model."},
 {'score': 0.014829578809440136,
  'token': 2846,
  'token_str': 'difficult',
  'sequence': "hello i'm a difficult model."},
 {'score': 0.014489353634417057,
  'token': 1266,
  'token_str': 'family',
  'sequence': "hello i'm a family model."}]

# 의학용어 전문 Corpus (EN -> KR)

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
corpus = pd.read_csv("/content/drive/MyDrive/텍스트 모델/Data/diagnosis_corpus.csv")

In [ ]:
corpus

,영문명,한글명
0,(Abt-)Letterer-Siwe disease,(Abt-)레테러-시웨병
1,(congenital) Mastocytosis(cutaneous),(선천성)비만세포증(피부)
2,(Congenital) Ota naevus,(선천성) 오타 모반
3,(Foster-)Kennedy syndrome,(포스터-)케네디 증후군
4,(Middle yin stage exuberant yin repelling yang...,소음음성격양증
...,...,...
198846,Μv,마이크로볼트
198847,Μw,마이크로와트
198848,Μμ,마이크로마이크론
198849,Μμg,마이크로마이크로그램


In [ ]:
corpus["영문명"][0]

'(Abt-)Letterer-Siwe disease'

In [ ]:
corpus["한글명"][0]

'(Abt-)레테러-시웨병'

In [ ]:
courpus_1=corpus.dropna()

In [ ]:
courpus_eng_list = courpus_1["영문명"].values.tolist()

In [ ]:
tokenized_data = tokenizer(courpus_eng_list, truncation=True, padding=True)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [ ]:
courpus_1["한글명"].values

array(['(Abt-)레테러-시웨병', '(선천성)비만세포증(피부)', '(선천성) 오타 모반', ..., '마이크로마이크론',
       '마이크로마이크로그램', '미세침윤암종, 자궁경부'], dtype=object)

In [ ]:
labels = courpus_1["한글명"].values
print(labels)

['(Abt-)레테러-시웨병' '(선천성)비만세포증(피부)' '(선천성) 오타 모반' ... '마이크로마이크론'
 '마이크로마이크로그램' '미세침윤암종, 자궁경부']


# 약어 해체 전문 Corpus (SF -> LF)

In [8]:
import pandas as pd
import numpy as np

In [32]:
corpus = pd.read_csv("/content/drive/MyDrive/텍스트 모델/Data/Acronym_metainventory2.csv")

In [33]:
corpus

,SF,LF
0,AA,achievement age
1,AA,Alcoholics Anonymous
2,AA,alcohol abuse
3,AA,alcohol-abuse
4,AA,aortic aneurysm
...,...,...
405538,VAVD,Vacuum-assisted vaginal delivery
405539,VBAC,Vaginal birth after caesarean section
405540,VSD,Ventricular septal defect
405541,VTOP,Voluntary termination of pregnancy


In [34]:
corpus["SF"][0]

'AA'

In [35]:
corpus["LF"][0]

'achievement age'

In [36]:
courpus_1=corpus.dropna()

In [37]:
print(courpus_1)

          SF                                     LF
0         AA                        achievement age
1         AA                   Alcoholics Anonymous
2         AA                          alcohol abuse
3         AA                          alcohol-abuse
4         AA                        aortic aneurysm
...      ...                                    ...
405538  VAVD       Vacuum-assisted vaginal delivery
405539  VBAC  Vaginal birth after caesarean section
405540   VSD              Ventricular septal defect
405541  VTOP     Voluntary termination of pregnancy
405542   VTX                                 Vertex

[405446 rows x 2 columns]


In [38]:
courpus_eng_list = courpus_1["LF"].values.tolist()

In [39]:
tokenized_data = tokenizer(courpus_eng_list, truncation=True, padding=True)

In [40]:
courpus_1["LF"].values

array(['achievement age', 'Alcoholics Anonymous', 'alcohol abuse', ...,
       'Ventricular septal defect', 'Voluntary termination of pregnancy',
       'Vertex'], dtype=object)

In [41]:
labels = courpus_1["LF"].values
print(labels)

['achievement age' 'Alcoholics Anonymous' 'alcohol abuse' ...
 'Ventricular septal defect' 'Voluntary termination of pregnancy' 'Vertex']


# 번역 모델 생성기 (각 모델당 10,000단어 학습) Ex) 의학용어 전문 Corpus

In [ ]:
%cd Korean-English-translator/

[Errno 2] No such file or directory: 'Korean-English-translator/'
/content/Korean-English-translator


In [ ]:
from langdetect import detect
import re
import os
import matplotlib.ticker as ticker
import matplotlib.pyplot as plt
import datetime
plt.style.use('seaborn')
import operator


from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model, Sequential, load_model
from tensorflow.keras.layers import Input, LSTM, Dense, GRU, Embedding, Layer, RepeatVector, TimeDistributed
from tensorflow.keras.losses import SparseCategoricalCrossentropy, CategoricalCrossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import plot_model, to_categorical
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard
import pydot

from src.helpme_l2_drop import *

In [ ]:
df_test = pd.read_csv('/content/drive/MyDrive/텍스트 모델/Data/diagnosis_corpus.csv')
df_test= df_test.dropna()
#model 19 만들기 = 180000:190000
#df_test = df_test[180000:190000]
#df_test = df_test[140000:150000]
df_test = df_test[100000:110000]


eng = preprocess(df_test['영문명'])
kor = preprocess(df_test['한글명'])

input_tensor, input_lang_tokenizer = tokenize(eng)
target_tensor, target_lang_tokenizer = tokenize(kor)

eng_test_vocab_size = len(input_lang_tokenizer.word_index)+1
kor_test_vocab_size = len(target_lang_tokenizer.word_index)+1

print(f'English vocab size: {eng_test_vocab_size}')
print(f'Korean vocab size: {kor_test_vocab_size}')

eng_test_max_length = len(input_tensor[0])
kor_test_max_length = len(target_tensor[0])

print(f'Longest English Sentence: {eng_test_max_length}')
print(f'Longest Korean Sentence: {kor_test_max_length}')

English vocab size: 5208
Korean vocab size: 7894
Longest English Sentence: 28
Longest Korean Sentence: 19


In [ ]:
eng_word_counts = input_lang_tokenizer.word_counts
kor_word_counts = target_lang_tokenizer.word_counts

sorted_eng_word_counts = sorted(eng_word_counts.items(), key=operator.itemgetter(1), reverse=True)
sorted_kor_word_counts = sorted(kor_word_counts.items(), key=operator.itemgetter(1), reverse=True)

top_20_eng_words = sorted_eng_word_counts[2:20]

top_20_kor_words = sorted_kor_word_counts[2:20]

In [ ]:
x, y = zip(*top_20_eng_words)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(input_tensor,target_tensor, test_size=.3)


In [ ]:
y_train_test = encode_output(y_train, kor_test_vocab_size)
y_test_test = encode_output(y_test, kor_test_vocab_size)

KeyboardInterrupt: ignored

In [ ]:
model = define_model(eng_test_vocab_size, kor_test_vocab_size, eng_test_max_length, kor_test_max_length, 50)
model.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['acc'])

In [ ]:
log_dir="logs/fit/"
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1, profile_batch=1000000)
filename = '/content/drive/MyDrive/텍스트 모델/model_test_19.h5'
checkpoint = ModelCheckpoint(filename, monitor='val_acc', verbose=1, save_best_only=True)


In [ ]:
history = model.fit(X_train, y_train_test, epochs=100, batch_size=20, validation_data=(X_test, y_test_test), callbacks=[checkpoint,tensorboard_callback], verbose=2)

In [ ]:
model.save('/content/drive/MyDrive/텍스트 모델/model_19.h5')

# 번역 모델 확인 - 단어 Ex) 의학용어 전문 Corpus

In [ ]:
n = int(100885)
#180031
#146260
print(df_test["영문명"][n],df_test["한글명"][n])

Mediastinum 종격


In [ ]:
df_test["영문명"][n]

'Mediastinum'

In [ ]:
df_test["한글명"][n]

'종격'

In [ ]:
decode_input(input_tensor,2000,input_lang_tokenizer)
sentence1 = df_test["영문명"][n]

test_sentence = encode_sentence(sentence1, input_lang_tokenizer,eng_test_max_length)

test_sentence.shape


(28,)

In [ ]:
print(test_sentence)

[  1 531   2   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0]


In [ ]:
model1 = tf.keras.models.load_model('/content/drive/MyDrive/텍스트 모델/model_1.h5')
model2 = tf.keras.models.load_model('/content/drive/MyDrive/텍스트 모델/model_2.h5')
model3 = tf.keras.models.load_model('/content/drive/MyDrive/텍스트 모델/model_3.h5')
model4 = tf.keras.models.load_model('/content/drive/MyDrive/텍스트 모델/model_4.h5')
model5 = tf.keras.models.load_model('/content/drive/MyDrive/텍스트 모델/model_5.h5')
model6 = tf.keras.models.load_model('/content/drive/MyDrive/텍스트 모델/model_6.h5')
model7 = tf.keras.models.load_model('/content/drive/MyDrive/텍스트 모델/model_7.h5')
model8 = tf.keras.models.load_model('/content/drive/MyDrive/텍스트 모델/model_8.h5')
model9 = tf.keras.models.load_model('/content/drive/MyDrive/텍스트 모델/model_9.h5')
model10 = tf.keras.models.load_model('/content/drive/MyDrive/텍스트 모델/model_10.h5')
model11 = tf.keras.models.load_model('/content/drive/MyDrive/텍스트 모델/model_11.h5')
model12 = tf.keras.models.load_model('/content/drive/MyDrive/텍스트 모델/model_12.h5')
model13 = tf.keras.models.load_model('/content/drive/MyDrive/텍스트 모델/model_13.h5')
model14 = tf.keras.models.load_model('/content/drive/MyDrive/텍스트 모델/model_14.h5')
model15 = tf.keras.models.load_model('/content/drive/MyDrive/텍스트 모델/model_15.h5')
model16 = tf.keras.models.load_model('/content/drive/MyDrive/텍스트 모델/model_16.h5')
model17 = tf.keras.models.load_model('/content/drive/MyDrive/텍스트 모델/model_17.h5')
model18 = tf.keras.models.load_model('/content/drive/MyDrive/텍스트 모델/model_18.h5')
model19 = tf.keras.models.load_model('/content/drive/MyDrive/텍스트 모델/model_19.h5')

In [ ]:
predict_sequence(model11,target_lang_tokenizer,test_sentence)
#predict_sequence(model15,target_lang_tokenizer,test_sentence)
#predict_sequence(model18,target_lang_tokenizer,test_sentence)
#모델 속 답이 없는 경우 (ex model18) ValueError 발생함

'<start> 종격'

In [ ]:
for i in range(2):
  try:
    a = predict_sequence(model14,target_lang_tokenizer,test_sentence)
    b = "try"
  except ValueError:
    a = predict_sequence(model15,target_lang_tokenizer,test_sentence)
    b = "except"
print(b, a)

except <start> 추정하다


In [ ]:
a_090 = predict_sequence(model19,target_lang_tokenizer,test_sentence)
print(a_090)

<start> 상승제


In [ ]:
list1 = [model1, model2, model3, model4, model5, model6, model7, model8, model9, model10, model11, model2, model13, model4, model15, model16, model17, model18, model19]
while True:
  for i in list1:
    try:
        predict_sequence(i,target_lang_tokenizer,test_sentence)
        a = predict_sequence(i,target_lang_tokenizer,test_sentence)
        break
    except ValueError:
      list1.remove(i)
      continue
print(a)

KeyboardInterrupt: ignored

In [ ]:
while True:
    try:
        a = predict_sequence(model1,target_lang_tokenizer,test_sentence)
        break
    except ValueError:
      try:
        a = predict_sequence(model2,target_lang_tokenizer,test_sentence)
        break
      except ValueError:
        try:
          a =predict_sequence(model3,target_lang_tokenizer,test_sentence)
          break
        except ValueError:
          try:
            a =predict_sequence(model4,target_lang_tokenizer,test_sentence)
            break
          except ValueError:
            try:
              a =predict_sequence(model5,target_lang_tokenizer,test_sentence)
              break
            except ValueError:
              try:
                a =predict_sequence(model6,target_lang_tokenizer,test_sentence)
                break
              except ValueError:
                try:
                  a =predict_sequence(model7,target_lang_tokenizer,test_sentence)
                  break
                except ValueError:
                  try:
                    a =predict_sequence(model8,target_lang_tokenizer,test_sentence)
                    break
                  except ValueError:
                    try:
                      a =predict_sequence(model9,target_lang_tokenizer,test_sentence)
                      break
                    except ValueError:
                      try:
                        a =predict_sequence(model10,target_lang_tokenizer,test_sentence)
                        break
                      except ValueError:
                        try:
                          a =predict_sequence(model11,target_lang_tokenizer,test_sentence)
                          break
                        except ValueError:
                          try:
                            a =predict_sequence(model12,target_lang_tokenizer,test_sentence)
                            break
                          except ValueError:
                            try:
                              a =predict_sequence(model13,target_lang_tokenizer,test_sentence)
                              break
                            except ValueError:
                              try:
                                a =predict_sequence(model14,target_lang_tokenizer,test_sentence)
                                break
                              except ValueError:
                                try:
                                  a =predict_sequence(model15,target_lang_tokenizer,test_sentence)
                                  break
                                except ValueError:
                                  try:
                                    a =predict_sequence(model16,target_lang_tokenizer,test_sentence)
                                    break
                                  except ValueError:
                                    try:
                                      a =predict_sequence(model17,target_lang_tokenizer,test_sentence)
                                      break
                                    except ValueError:
                                      try:
                                        a =predict_sequence(model18,target_lang_tokenizer,test_sentence)
                                        break
                                      except ValueError:
                                          a =predict_sequence(model19,target_lang_tokenizer,test_sentence)
                                          break
print(a)

InvalidArgumentError: ignored

# 번역 모델 확인 - 문장 Ex) 의학용어 전문 Corpus

In [ ]:
text = 'Lung, right lower lobe, "RLL wedge", wedge resection: 1. Chronic granulomatous inflammation (see note) 2. No evidence of neoplasm Pleura, "Pleural nodule", resection: Mature adipose tissue with no tumor'

In [ ]:
df_test["한글명"][182000]

'종말핵'

In [ ]:
df_test["영문명"][182000]

'Terminal nucleus'

In [ ]:
text = "Terminal nucleus is painful and I have diabetes"

In [ ]:
text = "I have problem with my lung. Moreover I have  which makes me hard everyday. Finally Tonic extension has problems"

In [ ]:
a1 = translate("English (en)","Korean (ko)",text)
print(a1)

/usr/local/lib/python3.8/dist-packages/transformers/generation/utils.py:1387: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 200 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


나는 내 폐에 문제가있다.또한 나는 매일 나를 힘들게 만듭니다.마침내 Tonic 확장 문제가있다.


In [ ]:
print(a1[0],a1[1],a1[2],a1[3],a1[4],a1[5])

나 는   내   폐


In [ ]:
#li1 = list(a1.split(" ;,."))
#li1 = [x.strip(" ") for x in a1.split('.')]
li1 = re.split("\s|(?<!\d)[,.](?!\d)", a1)
print(li1)

['나는', '내', '폐에', '문제가있다', '또한', '나는', '매일', '나를', '힘들게', '만듭니다', '마침내', 'Tonic', '확장', '문제가있다', '']


In [ ]:
a1

'나는 내 폐에 문제가있다.또한 나는 매일 나를 힘들게 만듭니다.마침내 Tonic 확장 문제가있다.'

In [ ]:
a2=a1
a2

'나는 내 폐에 문제가있다.또한 나는 매일 나를 힘들게 만듭니다.마침내 Tonic 확장 문제가있다.'

In [ ]:
# content_text = re.sub('[|ㄱ-ㅎ|ㅏ-ㅣ]+','',a2)
# a2 = a2.replace('"', ' ')
content_text = re.sub('[가-힣]+','',a2)
content_text = re.sub('[|0-9|]+','',content_text)
content_text = re.sub('[-=+,#/\?:^.@*\"※~ㆍ!』‘|\(\)\[\]`\'…》\”\“\’·]',' ',content_text)
print(content_text)

           Tonic   


In [ ]:
a = content_text.strip()
a

'Tonic'

In [ ]:
a3 = [c.strip() for c in content_text.split(' ') if c != '']
a3

#Use this to extract untranslated word
#Then translate the untranslated word using 의학용어 전문 Corpus + 번역 모델 생성기
#Finally replace the translated result with untranslated word

['Tonic']

# 번역 먼저 하고 Hugging face 돌리기 - X (우리가 만든 번역기에 일부 단어 없어서 번역 불가)

In [ ]:
for i in range(len(text_1)):
  decode_input(input_tensor,2000,input_lang_tokenizer)
  sentence1 = text_1a[i]

  test_sentence = encode_sentence(sentence1, input_lang_tokenizer,eng_test_max_length)

  test_sentence.shape

  while True:
    try:
        a = predict_sequence(model1,target_lang_tokenizer,test_sentence)
        break
    except ValueError:
      try:
        a = predict_sequence(model2,target_lang_tokenizer,test_sentence)
      except ValueError:
        try:
          a =predict_sequence(model3,target_lang_tokenizer,test_sentence)
        except ValueError:
          try:
            a =predict_sequence(model4,target_lang_tokenizer,test_sentence)
          except ValueError:
            try:
              a =predict_sequence(model5,target_lang_tokenizer,test_sentence)
            except ValueError:
              try:
                a =predict_sequence(model6,target_lang_tokenizer,test_sentence)              

              except ValueError:
                try:
                  a =predict_sequence(model7,target_lang_tokenizer,test_sentence)

                except ValueError:
                  try:
                    a =predict_sequence(model8,target_lang_tokenizer,test_sentence)

                  except ValueError:
                    try:
                      a =predict_sequence(model9,target_lang_tokenizer,test_sentence)

                    except ValueError:
                      try:
                         a =predict_sequence(model10,target_lang_tokenizer,test_sentence)

                      except ValueError:
                        try:
                          a =predict_sequence(model11,target_lang_tokenizer,test_sentence)

                        except ValueError:
                          try:
                            a =predict_sequence(model12,target_lang_tokenizer,test_sentence)

                          except ValueError:
                            try:
                              a =predict_sequence(model13,target_lang_tokenizer,test_sentence)

                            except ValueError:
                              try:
                                a =predict_sequence(model14,target_lang_tokenizer,test_sentence)

                              except ValueError:
                                try:
                                  a =predict_sequence(model15,target_lang_tokenizer,test_sentence)

                                except ValueError:
                                  try:
                                    a =predict_sequence(model16,target_lang_tokenizer,test_sentence)

                                  except ValueError:
                                    try:
                                      a =predict_sequence(model17,target_lang_tokenizer,test_sentence)

                                    except ValueError:
                                      try:
                                        a =predict_sequence(model18,target_lang_tokenizer,test_sentence)

                                      except ValueError:
                                          a =predict_sequence(model19,target_lang_tokenizer,test_sentence)
                                          break
          


KeyError: ignored